In [1]:
import os
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
from torchvision.utils import save_image, make_grid
from datasets import load_dataset
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
from pathlib import Path

In [2]:
checkpoint_path = "/home/hoffman/Documents/git/rectified-flow-pytorch/checkpoints/checkpoint.41000.pt"

In [3]:

class OxfordFlowersDataset(Dataset):
    def __init__(
        self,
        image_size
    ):
        self.ds = load_dataset('nelorth/oxford-flowers')['train']

        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.PILToTensor()
        ])

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        pil = self.ds[idx]['image']
        tensor = self.transform(pil)
        return tensor / 255.
    
    


flowers_dataset = OxfordFlowersDataset(
    image_size = 128
)

In [4]:
def download_checkpoint(hf_repo_id, checkpoint_path, local_path):
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    
    # Check if file already exists
    if os.path.exists(local_path):
        print(f"Checkpoint already exists at {local_path}")
        return local_path
    
    # Construct URL for the file
    url = f"https://huggingface.co/{hf_repo_id}/resolve/main/{checkpoint_path}"
    
    # Download the file
    print(f"Downloading checkpoint from {url}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    # Save the file
    with open(local_path, 'wb') as f:
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        for data in tqdm(response.iter_content(block_size), total=total_size//block_size, unit='KB'):
            f.write(data)
    
    print(f"Checkpoint downloaded to {local_path}")
    return local_path

In [5]:
checkpoint_path = download_checkpoint(
        hf_repo_id="Aadharsh/oxford-rf", 
        checkpoint_path="checkpoints/checkpoint.41000.pt",
        local_path="./checkpoints/checkpoint.41000.pt"
    )



# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load checkpoint
print("Loading checkpoint...")
checkpoint = torch.load(checkpoint_path, map_location=device)

Checkpoint already exists at ./checkpoints/checkpoint.41000.pt
Using device: cpu
Loading checkpoint...


/home/hoffman/anaconda3/envs/thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/tmp/ipykernel_811838/2449326727.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowe

In [6]:
from rectified_flow_pytorch import RectifiedFlow, Unet, Trainer

model = Unet(
    dim = 64,
    mean_variance_net = False
)

rectified_flow = RectifiedFlow(model)
rectified_flow.load_state_dict(checkpoint['model'])


<All keys matched successfully>

In [7]:
num_samples = 4
sample_indices = torch.randint(0, len(flowers_dataset), (num_samples,))
sample_images = [flowers_dataset[idx.item()] for idx in sample_indices]
images_batch = torch.stack(sample_images).to(device)

In [8]:
with torch.no_grad():
    noise = rectified_flow.reverse(images_batch)

In [ ]:
mean = x_flat.mean(dim=0)  # shape: (3,)


torch.Size([4, 3, 128, 128])

In [11]:
output_dir = "./reversed_flow_results"
os.makedirs(output_dir, exist_ok=True)
img_grid = make_grid(noise, nrow=2)
save_image(img_grid, f"{output_dir}/noise.png")

In [35]:
data_shape = images_batch.shape[1:]

with torch.no_grad():
    sampled = rectified_flow.sample(images_batch.shape[0], data_shape=data_shape)

In [37]:
from torchvision.utils import save_image

In [41]:
sampled = rearrange(sampled, '(row col) c h w -> c (row h) (col w)', row = 2)
sampled.clamp_(0., 1.)

save_image(sampled, "something.png")

In [39]:
from einops import einsum, reduce, rearrange, repeat
from einops.layers.torch import Rearrange

In [ ]:
eval_model = default(self.ema_model, self.model)
dl = cycle(self.dl)
mock_data = next(dl)
data_shape = mock_data.shape[1:]

with torch.no_grad():
    sampled = eval_model.sample(batch_size=self.num_samples, data_shape=data_shape)

sampled = rearrange(sampled, '(row col) c h w -> c (row h) (col w)', row = self.num_sample_rows)
sampled.clamp_(0., 1.)

save_image(sampled, fname)
return sampled

In [13]:
pip install torchdiffeq einx hyper_connections ema_pytorch


Note: you may need to restart the kernel to use updated packages.
